# Integration of Atos with Qiskit Runtime Sampler

In [1]:
import time
from qat.interop.qiskit import qlm_to_qiskit
from qat.lang.AQASM import Program
from qat.core.wrappers import Batch
from qat.lang.AQASM.gates import *

from qat.interop.qiskit import BackendToQPU
from qat.interop.qiskit import AsyncBackendToQPU
from qiskit import *

In [2]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
provider = IBMQ.get_provider(group='open')

## Define circuit

In [4]:
nbqubits = 5

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])
prog.apply(CNOT, qreg[0], qreg[2])
prog.apply(CNOT, qreg[0], qreg[3])
#prog.apply(CNOT, qreg[0], qreg[4])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

# Conversion
qiskit_circuit = qlm_to_qiskit(qlm_circuit)
print(qiskit_circuit)

      ┌───┐                     ┌─┐   
q0_0: ┤ H ├──■────■───────■─────┤M├───
      └───┘┌─┴─┐  │  ┌─┐  │     └╥┘   
q0_1: ─────┤ X ├──┼──┤M├──┼──────╫────
           └───┘┌─┴─┐└╥┘  │  ┌─┐ ║    
q0_2: ──────────┤ X ├─╫───┼──┤M├─╫────
                └───┘ ║ ┌─┴─┐└╥┘ ║ ┌─┐
q0_3: ────────────────╫─┤ X ├─╫──╫─┤M├
       ┌─┐            ║ └───┘ ║  ║ └╥┘
q0_4: ─┤M├────────────╫───────╫──╫──╫─
       └╥┘            ║       ║  ║  ║ 
c0: 5/══╩═════════════╩═══════╩══╩══╩═
        4             1       2  0  3 


## Blocking call to Runtime

In [5]:
qlm_job = qlm_circuit.to_job(nbshots=1024)

# Runtime using ibmq_qasm_simulator
backend = provider.get_backend('ibmq_qasm_simulator')
qpu = BackendToQPU(backend)

# Execute via Qiskit Runtime
result = qpu.submit(qlm_job)
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))

State: |01111>	 probability: 0.4990234375
State: |00000>	 probability: 0.5009765625


## Async call to Runtime

In [13]:
# Make Async QPU
async_qpu = AsyncBackendToQPU(backend)

# Async execution
async_result = async_qpu.submit(qlm_job)

# Wait for the result
while not async_result.result():
    time.sleep(1)

# Get result
result = async_result.result()
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))

State: |00000>	 probability: 0.4912109375
State: |01111>	 probability: 0.5087890625


## Batch test

In [7]:
nbqubits = 2
prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

qlm_job1 = qlm_circuit.to_job(nbshots=1024)
qlm_job2 = qlm_circuit.to_job(nbshots=1024)
batch = Batch(jobs=[qlm_job1, qlm_job2])

results = qpu.submit(batch)

In [10]:
for res in results:
    for entry in res.raw_data:
        print("State: {}\t probability: {}".format(entry.state, entry.probability))
    print()

State: |11>	 probability: 0.5146484375
State: |00>	 probability: 0.4853515625

State: |00>	 probability: 0.5029296875
State: |11>	 probability: 0.4970703125



## Async even superposition test

In [16]:
nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(H, qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

qlm_job = qlm_circuit.to_job(nbshots=1024)

async_job = async_qpu.submit_job(qlm_job)

loop_nb = 0
while async_job.result() is None:
    time.sleep(5)
    loop_nb += 1
    if loop_nb > 4:
        break

result = async_job.result()

for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))

State: |01>	 probability: 0.2626953125
State: |11>	 probability: 0.2470703125
State: |00>	 probability: 0.2412109375
State: |10>	 probability: 0.2490234375
